# Implementation part 2

In [3]:
import os
import tifffile as tif
import cv2
import numpy as np
from src.controllers.utility import compute_spline

with tif.TiffFile(os.getcwd() + r"\test_data\Expansion dSTORM-Line Profile test.tif") as file:
    image = file.asarray().astype(np.uint8)*40

We run the steps from part 1 with the helper function compute spline and receive a `CustomSpline` object. Using the `sample` function with the number of points gives us the center points of our spline.

In [5]:
spline = compute_spline(image, blur=20, )[0]
sample = spline.sample(spline.n_points)

0
1
2
3
4
5
6


To fit the results into a bimodal gaussian we just have to sum two of those functions with independent parameters. To only apply a background once we set B=0 within the distributions and add an additional B to our bigaussian model.

In [ ]:
def gaussian(x, I, sig, c, B):
    return I* np.exp(-(x - c) ** 2 / (2 * sig** 2)) + B


def bigaussian(x, I1, sig1, c1, I2, sig2, c2, B):
    return (gaussian(x, I1, sig1, c1, 0) +
            gaussian(x, I2, sig2, c2, 0) + B)

# Fit data to the defined function:
Fitting a function to data in python is quite easy since (like for kinda everything else) there were already some people who did the work for us. We will use scipy.optimize to do the job.
The estimization of initial parameters is crucial for a good fit. Therefore, we estimate a first guess for our parameters and bind the scipy optimization to stay within reasonable bounds.
The first thing we require is that all our parameters are >0 that's why we set the first value of every tuple to 0. The intensity I is constraint with an upper bound of the datas maximum value. We basically don't care what our sigma is doing and set the maximum to np.inf . However we want our center to stay within the bounds of our line profile.

In [ ]:
def bounds(data):
    return np.array([[0, data.max() + 0.1], [0, np.inf], [0, data.shape[0]],
                    [0, data.max() + 0.1], [0, np.inf], [0, data.shape[0]],
                     [0, 0.1]]).T

def guess(data):
    return [data.max()/2, 0.5, np.where(data==data.max())[0],
            data.max()/2, 0.5, np.where(data==data.max())[0], 0]

In [ ]:
def fit_data_to(func, x, data):
    # calculate error by squared distance to data
    errfunc = lambda p, x, y: (func(x, *p) - y) ** 2

    result = optimize.least_squares(errfunc, guess(data), bounds=bounds(data), args=(x, data))
    optim = result.x
    return optim

In [ ]:
x = np.arange(0,interpolated_profile.shape[0],1)
fig = plt.figure()
ax1 = fig.add_axes((0.1, 0.2, 0.8, 0.7))
ax1.plot(x, interpolated_profile / interpolated_profile.max(), c="r", label="averaged line profile")

optim = fit_data_to(bigaussian, x, interpolated_profile)
ax1.plot(x, bigaussian(x, *optim) / interpolated_profile.max(),
         lw=1, c="b", ls='--', label="bigaussian fit")
ax1.legend(loc='best')
ax1.set_ylabel("normed intensity [a.u.]")
ax1.set_xlabel("distance [nm]")
plt.show()